# Agents as Tools with Strands Agents / 使用 Strands Agents 的代理作为工具


"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.

"代理作为工具"是 AI 系统中的一种架构模式，其中专门的 AI 代理被包装为可调用的函数（工具），可供其他代理使用。这创建了一个分层结构：

1. 主要的"编排器"代理处理用户交互并确定调用哪个专门代理

2. 专门的"工具代理"在被编排器调用时执行特定领域的任务

这种方法模仿了人类团队动态，其中经理协调专家，每个人都带来独特的专业知识来解决复杂问题。与单个代理试图处理所有事情不同，任务被委派给最合适的专门代理。



## Key Benefits and Core Principles / 关键优势和核心原则

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task

"代理作为工具"模式提供了几个优势：

- 关注点分离：每个代理都有专注的责任领域，使系统更容易理解和维护
- 分层委派：编排器决定调用哪个专家，创建清晰的指挥链
- 模块化架构：专家可以独立添加、删除或修改，而不影响整个系统
- 性能改进：每个代理都可以有针对其特定任务优化的定制系统提示和工具


In [3]:
import os

from strands import Agent, tool
from strands_tools import file_write


In this module we will be creating an orchestrator based multi-agent workflow. We will also explore `use_llm` which allows use to create nested agents.

在本模块中，我们将创建基于编排器的多代理工作流。我们还将探索 `use_llm`，它允许我们创建嵌套代理。

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>





## Research Agent / 研究代理

Lets first create a basic reasearch assistant with http_request tool.

让我们首先创建一个带有 http_request 工具的基本研究助手。

In [1]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [4]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

# Amazon Bedrock Overview

Amazon Bedrock is AWS's fully managed service that provides access to foundation models (FMs) through a unified API. Launched in general availability in September 2023, it enables organizations to build generative AI applications without the complexity of deploying and managing the underlying infrastructure.

## Key Features

### Model Access and Selection
- **Multiple Foundation Models**: Offers access to models from AI21 Labs, Anthropic, Cohere, Meta, Mistral AI, Stability AI, and Amazon's own models (Amazon Titan)[^1][^2]
- **Model Evaluation**: Tools to compare and evaluate different models for specific use cases
- **Serverless Experience**: Pay-only-for-what-you-use pricing model with no infrastructure management[^3]

### Security and Privacy
- **Private Customization**: Custom models remain private within your AWS account
- **Data Protection**: Your data is not used to train base models
- **Enterprise Security**: Supports VPC endpoints, AWS KMS encrypti

Now we can wrap this agent as a tool. Allowing other agents to interact with it. 

####  Best Practices for Agent as Tools

When implementing the "Agents as Tools" pattern with Strandly AI:

1. Clear tool documentation: Write descriptive docstrings that explain the agent's expertise
2. Focused system prompts: Keep each specialized agent tightly focused on its domain
3. Proper response handling: Use consistent patterns to extract and format responses
4. Tool selection guidance: Give the orchestrator clear criteria for when to use each specialized agen

现在我们可以将此代理包装为工具。允许其他代理与其交互。

#### 代理作为工具的最佳实践

在使用 Strandly AI 实现"代理作为工具"模式时：

1. 清晰的工具文档：编写描述性的文档字符串来解释代理的专业知识
2. 专注的系统提示：保持每个专门代理紧密专注于其领域
3. 适当的响应处理：使用一致的模式来提取和格式化响应
4. 工具选择指导：为编排器提供何时使用每个专门代理的明确标准

In [5]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

Now lets follow the best practices and create `product_recommendation_assistant`, `trip_planning_assistant`, and `orchestrator` agent.

现在让我们遵循最佳实践，创建 `product_recommendation_assistant`、`trip_planning_assistant` 和 `orchestrator` 代理。

### Product Recommendation Assistant / 产品推荐助手

In [6]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [7]:
product_recommendation_assistant("Product recommendations for flying cars")

# Product Recommendations for Flying Cars

While true flying cars aren't yet commercially available for general consumers, there are several advanced projects and pre-order options in development. Here are some recommendations based on current information:

## Current Projects with Pre-Order Options

1. **PAL-V Liberty** - The world's first commercially available flying car with road and air certification processes underway. Offers a gyrocopter-style flight with prices starting around $400,000-600,000.
   [Source: PAL-V official website]

2. **AeroMobil 4.0** - A Slovak hybrid car-aircraft with folding wings, targeting wealthy buyers and flight enthusiasts. Estimated price: $1.2-1.6 million.
   [Source: AeroMobil website]

3. **Terrafugia Transition** - A roadable aircraft that can switch between flying and driving modes, owned by Geely. Reservations available with estimated $300,000-400,000 price tag.
   [Source: Terrafugia corporate information]

## For Aviation Enthusiasts (Availabl

"# Product Recommendations for Flying Cars\n\nWhile true flying cars aren't yet commercially available for general consumers, there are several advanced projects and pre-order options in development. Here are some recommendations based on current information:\n\n## Current Projects with Pre-Order Options\n\n1. **PAL-V Liberty** - The world's first commercially available flying car with road and air certification processes underway. Offers a gyrocopter-style flight with prices starting around $400,000-600,000.\n   [Source: PAL-V official website]\n\n2. **AeroMobil 4.0** - A Slovak hybrid car-aircraft with folding wings, targeting wealthy buyers and flight enthusiasts. Estimated price: $1.2-1.6 million.\n   [Source: AeroMobil website]\n\n3. **Terrafugia Transition** - A roadable aircraft that can switch between flying and driving modes, owned by Geely. Reservations available with estimated $300,000-400,000 price tag.\n   [Source: Terrafugia corporate information]\n\n## For Aviation Enthu

### Trip Planning Assistant / 旅行规划助手

In [8]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator Agent / 编排器代理

In [9]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [10]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [11]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

I'll help you find hiking boot recommendations, and I'll write the response to a file in the current directory. Let me handle that for you.
Tool #1: product_recommendation_assistant
# Hiking Boot Recommendations

Based on your interest in hiking boots, here are some well-regarded options across different price points:

## Entry-Level Options
- **Merrell Moab 2 Mid Waterproof** ($145) - Often called "Mother of All Boots," known for out-of-the-box comfort and good water resistance
- **Columbia Newton Ridge Plus II** ($90) - Affordable waterproof leather boots with good traction

## Mid-Range Quality
- **Salomon X Ultra 3 Mid GTX** ($165) - Lightweight with excellent stability and GORE-TEX waterproofing
- **Keen Targhee III Mid** ($175) - Wide toe box, good ankle support, and waterproof membrane

## Premium Options
- **Lowa Renegade GTX Mid** ($245) - Versatile all-terrain boots with exceptional durability and comfort
- **Scarpa Zodiac Plus GTX** ($299) - Superior craftsmanship with excel

╔════════ File Write Operation ════════╗
║                                      ║
║ Path: hiking_boot_recommendations.md ║
║ Size: 1003 characters                ║
║                                      ║
╚══════════════════════════════════════╝

╔═════════════════════ Write Successful ══════════════════════╗
║ File written successfully to hiking_boot_recommendations.md ║
╚═════════════════════════════════════════════════════════════╝

I've written hiking boot recommendations to a file named "hiking_boot_recommendations.md" in your current directory. The file includes a range of options across different price points from entry-level to premium boots.

The recommendations include popular models like the Merrell Moab 2, Columbia Newton Ridge Plus II, Salomon X Ultra 3, Keen Targhee III, Lowa Renegade GTX, and Scarpa Zodiac Plus GTX, along with their approximate prices and key features.

If you need more specific recommendations based on your hiking style, terrain preferences, budget, or foot-specific needs, just let me know!

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

让我们看看编排器的消息。在这里您可以看到代理决定使用子代理作为工具

In [12]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find hiking boot recommendations, and I'll write the response to a file in the current directory. Let me handle that for you."},
   {'toolUse': {'toolUseId': 'tooluse_Xy1FrFvIT_Os9zbJCAHa0g',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots. Please provide recommendations for good hiking boots."}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_Xy1FrFvIT_Os9zbJCAHa0g',
     'status': 'success',
     'content': [{'text': '# Hiking Boot Recommendations\n\nBased on your interest in hiking boots, here are some well-regarded options across different price points:\n\n## Entry-Level Options\n- **Merrell Moab 2 Mid Waterproof** ($145) - Often called "Mother of All Boots," known for out-of-the-box comfort and good water resistance\n- 

In [13]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

I'd be happy to help you plan a trip to Patagonia! I'll use the trip planning assistant to create a detailed itinerary for you. However, I'd need some additional information to customize your trip properly.

Could you please provide some details about:
- When you're planning to visit (time of year and duration)
- Your travel style/preferences (luxury, budget, adventure, etc.)
- Any specific activities you're interested in (hiking, wildlife watching, photography, etc.)
- Any particular areas of Patagonia you want to visit (Chilean side, Argentinian side, or both)

This will help me create a more personalized Patagonia itinerary for you.

In [14]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find hiking boot recommendations, and I'll write the response to a file in the current directory. Let me handle that for you."},
   {'toolUse': {'toolUseId': 'tooluse_Xy1FrFvIT_Os9zbJCAHa0g',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots. Please provide recommendations for good hiking boots."}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_Xy1FrFvIT_Os9zbJCAHa0g',
     'status': 'success',
     'content': [{'text': '# Hiking Boot Recommendations\n\nBased on your interest in hiking boots, here are some well-regarded options across different price points:\n\n## Entry-Level Options\n- **Merrell Moab 2 Mid Waterproof** ($145) - Often called "Mother of All Boots," known for out-of-the-box comfort and good water resistance\n- 

### Calling multiple agents / 调用多个代理

In [15]:
orchestrator.messages = []

In [17]:
query = "Can you do a research on the south of Spain? Also help me plan a 3 day trip."

orchestrator(query)

I'd be happy to research the south of Spain and help you plan a 3-day trip to the region. Let me start with some research on southern Spain:
Tool #5: research_assistant
# Southern Spain (Andalusia): A Comprehensive Overview

## Geography and Location

Andalusia (Spanish: Andalucía) is Spain's southernmost region, covering 87,597 square kilometers (33,821 square miles) and constituting approximately 17% of Spain's total area. It's bordered by:
- Portugal to the west
- The Mediterranean Sea and Gibraltar to the south
- Murcia and Castilla-La Mancha to the east
- Extremadura and Castilla-La Mancha to the north[1]

The region features diverse geographical elements:
- The Guadalquivir River valley, forming a fertile basin
- Sierra Morena mountain range to the north
- The Baetic System, including Sierra Nevada (home to mainland Spain's highest peak, Mulhacén at 3,479 meters)
- 836 kilometers of coastline along the Atlantic Ocean and Mediterranean Sea[2]

## History

Andalusia possesses one o

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Based on the research and travel planning information, here\'s a comprehensive overview of Southern Spain and a tailored 3-day itinerary:\n\n# Southern Spain (Andalusia) Overview\n\n## Geography and Character\nAndalusia is Spain\'s southernmost region, covering 87,597 square kilometers (about 17% of Spain) with 836 kilometers of coastline along the Mediterranean Sea and Atlantic Ocean. The region features diverse geography including:\n- The fertile Guadalquivir River valley\n- Sierra Nevada mountains (with mainland Spain\'s highest peak)\n- Sierra Morena mountain range\n- Europe\'s warmest climate with over 300 sunny days per year along the coast\n\n## Rich Historical Heritage\nSouthern Spain has an incredibly layered history:\n- Ancient settlements by Phoenicians (who founded Cádiz around 1100 BCE)\n- Roman rule (206 BCE-5th century CE) when it was called "Baetica"\n- Islamic period (711-1492) when

Behind the scenes, the orchestrator will:
1. First call the `research_assistant`
2. Then call `trip_planning_assistant`
3. Combine these specialized responses into a cohesive answer that addresses both the queries

在幕后，协调器将：
1. 首先调用research_assistant
2. 然后调用trip_planning_assistant
3. 将这些专业的响应组合成一个连贯的答案，以解决这两个查询


### Sequential Agent Communication Pattern / 顺序代理通信模式 


The agent tool can also combine multiple agents together. In this example we will provide output of `research_agent` to `summary_agent` and return the summarized response.

代理工具也可以将多个代理组合在一起。在这个例子中，我们将把`research_agent`的输出提供给`summary_agent`，并返回总结的响应。

In [18]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

Multiple agents created successfully!

🔍 RESEARCH AGENT working on: generative Ai

# Generative AI: A Comprehensive Overview

Generative AI refers to artificial intelligence systems capable of creating new content, including text, images, audio, code, and other data types that resemble human-created content. Below is a comprehensive overview of this rapidly evolving field.

## Fundamental Concepts

Generative AI systems are primarily built on deep learning architectures, especially:

- **Large Language Models (LLMs)**: Neural networks trained on vast text corpora that can generate human-like text (e.g., GPT-4, LLaMA, PaLM)
- **Diffusion Models**: AI systems that learn to generate images by gradually removing noise from random patterns (e.g., Stable Diffusion, DALL-E)
- **Generative Adversarial Networks (GANs)**: Systems using two neural networks competing against each other to produce realistic outputs[1]

## Key Applications

Generative AI has found applications across numerous domain

## Congrats! / 恭喜！